In [24]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path

In [25]:
segment_path = Path("../../datasets/comma2k19/Chunk_1/b0c9d2329ad1606b|2018-08-02--08-34-47/40")

In [27]:
gps_times = np.load(segment_path / "global_pose/frame_gps_times")
gps_times

array([[  2012.      , 404106.397   ],
       [  2012.      , 404106.447008],
       [  2012.      , 404106.49699 ],
       ...,
       [  2012.      , 404166.246156],
       [  2012.      , 404166.296184],
       [  2012.      , 404166.34616 ]])

In [33]:
orientations = np.load(segment_path / "global_pose/frame_orientations")
orientations

array([[ 0.21243875, -0.80302968, -0.43522125, -0.34726874],
       [ 0.21230022, -0.80305676, -0.43500007, -0.34756783],
       [ 0.21228429, -0.80303141, -0.43482749, -0.34785196],
       ...,
       [ 0.22137891, -0.79545387, -0.42532217, -0.37060162],
       [ 0.22115694, -0.79572304, -0.42520901, -0.37028604],
       [ 0.22121323, -0.79582271, -0.425087  , -0.37017829]])

In [29]:

positions = np.load(segment_path / "global_pose/frame_positions")
positions

array([[-2712087.51680896, -4261670.05595519,  3881014.4539217 ],
       [-2712087.3712095 , -4261669.85464656,  3881014.76488791],
       [-2712087.22355988, -4261669.65116858,  3881015.07854078],
       ...,
       [-2711723.08011481, -4261177.54831335,  3881817.54470798],
       [-2711722.87567608, -4261177.28280163,  3881818.02584841],
       [-2711722.67333332, -4261177.02108768,  3881818.50131921]])

In [30]:

times = np.load(segment_path / "global_pose/frame_times")
times

array([46408.547498, 46408.597506, 46408.647488, ..., 46468.396654,
       46468.446682, 46468.496658])

In [31]:

velocities = np.load(segment_path / "global_pose/frame_velocities")
velocities

array([[2.9047239 , 4.01603002, 6.20555644],
       [2.94151362, 4.05735186, 6.25606847],
       [2.9777254 , 4.09487774, 6.30539503],
       ...,
       [4.10014952, 5.3344189 , 9.66065472],
       [4.06634053, 5.26615064, 9.55497136],
       [4.01985485, 5.19355877, 9.45527469]])

# GNSS

In [36]:
gnss_t = np.load(segment_path / "processed_log/GNSS/live_gnss_qcom/t")
gnss_t

array([46410.29684812, 46412.29723713, 46414.29308797, 46416.29803265,
       46418.29797005, 46420.30953093, 46422.29377942, 46424.2950413 ,
       46426.29160671, 46428.30367687, 46430.29929062, 46432.29663254,
       46434.28735718, 46436.29763671, 46438.3061253 , 46440.30276056,
       46442.33209535, 46444.29442504, 46446.29399394, 46448.30728665,
       46450.32744207, 46452.31319431, 46454.30869717, 46456.30338889,
       46458.30713555, 46460.30256243, 46462.29471451, 46464.30187071,
       46466.29722326, 46468.29711513])

In [38]:
gnss_value = np.load(segment_path / "processed_log/GNSS/live_gnss_qcom/value")
gnss_value

array([[ 3.77211070e+01, -1.22472312e+02,  7.42999983e+00,
         1.53322649e+12,  3.90000000e+01,  6.19999981e+00],
       [ 3.77213618e+01, -1.22472262e+02,  1.09600000e+01,
         1.53322649e+12,  3.80000000e+01,  7.19999981e+00],
       [ 3.77216432e+01, -1.22472240e+02,  1.44799995e+01,
         1.53322649e+12,  3.90000000e+01,  6.80000019e+00],
       [ 3.77219319e+01, -1.22472226e+02,  1.66200008e+01,
         1.53322650e+12,  3.40000000e+01,  6.19999981e+00],
       [ 3.77222951e+01, -1.22472221e+02,  2.00699997e+01,
         1.53322650e+12,  3.60000000e+01,  5.19999981e+00],
       [ 3.77226593e+01, -1.22472186e+02,  2.03600006e+01,
         1.53322650e+12,  3.70000000e+01,  6.40000010e+00],
       [ 3.77230520e+01, -1.22472166e+02,  2.04500008e+01,
         1.53322650e+12,  3.70000000e+01,  5.19999981e+00],
       [ 3.77233998e+01, -1.22472130e+02,  1.98299999e+01,
         1.53322650e+12,  3.80000000e+01,  5.59999990e+00],
       [ 3.77237253e+01, -1.22472126e+02,  1.912